# essayer la technique du web scrapping sur seulement une page

In [1]:
from scrapy import Selector
import requests

url = "https://media.ipsapps.org/shu/osa/bible/latin/01-GEN-001.html"
html = requests.get(url).content
sel = Selector(text=html)

# Get the text of the first paragraph
print(sel.css('span.button:nth-of-type(5)').extract())
num_page_suivant = sel.xpath('//span[@class="button"]/a/@href').extract()  
print(num_page_suivant)

['<span class="button"><a href="01-GEN-002.html" title="Next Chapter">\xa0\xa0&gt;\xa0\xa0</a></span>']
['00-INT2-000.html', '01-GEN-000.html', '01-GEN-002.html', '02-EXO-000.html']


# Essayons de recuperer les div sans ceux de classe s car ils contiennet des titres

In [ ]:
text = sel.css('div#content div.txs ::text').extract() # Pour récupérer le texte de la page
titres = sel.xpath('//div[contains(@id,"Ts")]/text()').extract()
print(text)
print(titres)
text = [t.strip() for t in text if t.strip()] # Pour enlever les espaces vides et les sauts de ligne ainsi que les balises html
titres = [t.strip() for t in titres if t.strip()]
#print(text)
for t in text:
    if t in titres:
        text.remove(t)
#premier = text[0].xpath('./div[@id="Ts1"]')
print(text)

list = [] # Pour stocker les versets
print(list)
i = 0
for t in text: # Pour parcourir les lignes de textes et supprimer les numeros.En gros c'est pour que les numeros des versets ne figurent pas dans ma liste des versets
    if len(t) <= 2:
        i+=1
    else:
        if len(list) == i+1:
            list[i] = list[i] + " " + t
        else:
            list.append(t)

for l in list:
    for caract in l:
        l = l.replace("«", "")
        l = l.replace("»", "")
    with open('dataset.txt', 'a+') as file :
            file.write(l)
            file.write("\n")

['Al‑khaliige', '1', '\xa0', 'Fi l‑bidaaya,', 'Allah khalag al‑samaawaat wa l‑ard.', '2', '\xa0', 'Wa l‑ard faadiye.', 'Hi ma indaha cheyy foogha.', 'Wa l‑dalaam gaaʼid yikhatti al‑almi al‑khariig wa Ruuh Allah gaaʼid yuhuum fi l‑almi.', '3', '\xa0', 'Wa Allah al‑Rabb gaal\u202f:', '«\u202fKhalli yukuun nuur.', '\u202f» Wa khalaas,', 'al‑nuur kaan.', '4', '\xa0', 'Wa Allah al‑Rabb chaaf kadar al‑nuur da sameh.', 'Wa farag al‑nuur min al‑dalaam.', '5', '\xa0', 'Wa samma al‑nuur nahaar wa l‑dalaam leel.', 'Wa kaan achiiye wa kaan fajur.', 'Da l‑yoom al‑awwal.', '6', '\xa0', 'Wa Allah al‑Rabb gaal\u202f:', '«\u202fKhalli tukuun faraga been al‑almi wa tifaarig al‑almi min al‑almi.\u202f»', '7', '\xa0', 'Wa l‑Rabb khalag al‑faraga wa farag beeha al‑almi al‑tihit min al‑almi al‑foog.', 'Wa khalaas,', 'bigi misil da.', '8', '\xa0', 'Wa samma al‑faraga di,', 'sama.', 'Wa kaan achiiye wa kaan fajur.', 'Da l‑yoom al‑taani.', '9', '\xa0', 'Wa l‑Rabb gaal\u202f:', '«\u202fKhalli al‑almi al‑tihit l

In [4]:
text = sel.css('div#content div.txs ::text') # Pour récupérer le texte de la page
print(text)
text = text.extract() # Pour extraire le texte
text = [t.strip() for t in text if t.strip()] # Pour enlever les espaces vides et les sauts de ligne ainsi que les balises html
print(text)
list = [] # Pour stocker les versets
print(list)
i = 0
for t in text: # Pour parcourir les lignes de textes et supprimer les numeros.En gros c'est pour que les numeros des versets ne figurent pas dans ma liste des versets
    if len(t) <= 2:
        i+=1
    else:
        if len(list) == i+1:
            list[i] = list[i] + " " + t
        else:
            list.append(t)

for l in list:
    for caract in l:
        l = l.replace("«", "")
        l = l.replace("»", "")
    with open('dataset.txt', 'a+') as file :
            file.write(l)
            file.write("\n")

[<Selector query="descendant-or-self::div[@id = 'content']/descendant-or-self::*/div[@class and contains(concat(' ', normalize-space(@class), ' '), ' txs ')]/descendant-or-self::text()" data='Al‑khaliige'>, <Selector query="descendant-or-self::div[@id = 'content']/descendant-or-self::*/div[@class and contains(concat(' ', normalize-space(@class), ' '), ' txs ')]/descendant-or-self::text()" data='1'>, <Selector query="descendant-or-self::div[@id = 'content']/descendant-or-self::*/div[@class and contains(concat(' ', normalize-space(@class), ' '), ' txs ')]/descendant-or-self::text()" data='\xa0'>, <Selector query="descendant-or-self::div[@id = 'content']/descendant-or-self::*/div[@class and contains(concat(' ', normalize-space(@class), ' '), ' txs ')]/descendant-or-self::text()" data='Fi l‑bidaaya,'>, <Selector query="descendant-or-self::div[@id = 'content']/descendant-or-self::*/div[@class and contains(concat(' ', normalize-space(@class), ' '), ' txs ')]/descendant-or-self::text()" data=

# La creation de mon arraigné ou plutot robots d'indexations

# Pour l'arabe

In [1]:
# Ici nous allons ecrire un code qui permet de recuperer toutes les pages du site web avec les methodes d'en haut
import scrapy, re
from scrapy.crawler import CrawlerProcess


class DCspider( scrapy.Spider ):  

    name = "dcspider"

    def start_requests( self ):    
        urls = "https://media.ipsapps.org/shu/osa/bible/latin/01-GEN-001.html"
        yield scrapy.Request( url = urls, callback = self.parse )

        
    def parse( self, response ): 
        num_page_suivant = response.xpath('//span[@class="button"]/a/@href').extract()[2]  
        link = "https://media.ipsapps.org/shu/osa/bible/latin/" + num_page_suivant
        titres = response.xpath('//div[contains(@id,"Ts")]/text()').extract()

        text = response.css('div#content div.txs ::text').extract() # Pour récupérer le texte de la page
        text = [t.strip() for t in text if t.strip()] # Pour enlever les espaces vides et les sauts de ligne ainsi que les balises html
        titres = [t.strip() for t in titres if t.strip()]
        #print("----------", num_page_suivant, "-----------------")
        if len(text) == 0:
            yield response.follow( url = link, callback = self.parse )

        for t in text: # je parcours ma liste pour supprimer les entetes cad les titres du chapitre
            if t in titres:
                text.remove(t)

        # Ecrivons une expression reguliere pour supprimer les numeros couplet  des versets
        #regex = re.compile(r"[0-9]+.[0-9]+")
        regex = r"\d{1,2}‑\d{1,2}"


        for l in text:
            l = l.replace("«", "")
            l = l.replace("»", "")
            l = l.replace("<", "")
            l = l.replace(">", "")
            l = l.replace("*", "")
            #print(l)

            if l.isnumeric() == True:
                l = l.replace(l, "\n")
            l = re.sub(regex, "\n\n", l)
            with open('arabe_tchad3.txt', 'a+') as file :
                    file.write(l)
        #print(text)
        yield response.follow( url = link, callback = self.parse )


process = CrawlerProcess() # Voila la derniere partie qui demarre le robot
process.crawl(DCspider)
process.start()


2025-05-04 13:42:27 [scrapy.utils.log] INFO: Scrapy 2.12.0 started (bot: scrapybot)
2025-05-04 13:42:27 [scrapy.utils.log] INFO: Versions: lxml 5.3.1.0, libxml2 2.12.9, cssselect 1.3.0, parsel 1.10.0, w3lib 2.3.1, Twisted 24.11.0, Python 3.12.3 (main, Feb  4 2025, 14:48:35) [GCC 13.3.0], pyOpenSSL 25.0.0 (OpenSSL 3.4.1 11 Feb 2025), cryptography 44.0.2, Platform Linux-6.11.0-17-generic-x86_64-with-glibc2.39
2025-05-04 13:42:27 [scrapy.addons] INFO: Enabled addons:
[]
2025-05-04 13:42:27 [scrapy.utils.log] DEBUG: Using reactor: twisted.internet.epollreactor.EPollReactor
2025-05-04 13:42:27 [scrapy.extensions.telnet] INFO: Telnet Password: 515e51ccd7cade98
2025-05-04 13:42:27 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.corestats.CoreStats',
 'scrapy.extensions.telnet.TelnetConsole',
 'scrapy.extensions.memusage.MemoryUsage',
 'scrapy.extensions.logstats.LogStats']
2025-05-04 13:42:27 [scrapy.crawler] INFO: Overridden settings:
{}
2025-05-04 13:42:28 [scrapy.middlewa

In [8]:
import re
#regex = re.compile(r"[0-9]+.[0-9]+")
regex = r"\d{1,2}‑\d{1,2}"

"""def chercher_format(liste):
    for chaine in liste:
        print(chaine)
        parties = chaine.split('‑')
        print(parties)
        if len(parties) == 2 and all(partie.isdigit() for partie in parties):
            print("ok")
            if 1 <= len(parties[0]) <= 2 and 1 <= len(parties[1]) <= 2:
                liste = liste.replace(chaine, "\n\n")
    return liste"""
# Exemple d'utilisation

text = ["Bariiʼa wa Chamaʼ humman kubaaraat al‑aayilaat al‑saakniin fi Ayyaluun.","Wa humman bas al‑tarado sukkaan Gaat.14‑16Wa awlaad Bariiʼa humman Akhyu wa Chaachakh wa Yarimuut wa Zabadya wa Araad wa Adaar wa Mikaayiil wa Yichfa wa Yuuha.","17‑18 Wa awlaad Alifaaʼal humman Zabadya wa Machullam wa Hizkhi wa Haabar wa Yichmaraay wa Yizliiya wa Yubaab.19‑21Wa awlaad Chimʼi humman Yakhiim wa Zikri wa Zabdi wa Aliʼinaay wa Siltaay wa Aliyiil wa Adaaya wa Baraaya wa Chimrat.22‑25Wa awlaad Chaachakh humman Yichfaan wa Ibaar wa Aliyiil wa Abduun wa Zikri wa Hanaan wa Hananya wa Ilaam wa Antuutiya wa Yifadya wa Fanuwiil.26‑27Wa awlaad Yaruuhaam humman Chamachraay wa Chaharya wa Atalya wa Yaʼrachya wa Iliiya wa Zikri."]
liste = []
for i in text:
    i = re.sub(regex, r'\n\n', i)
    liste.append(i)
print(liste)
#text = chercher_format(text)
print(text)
"""with open('arabe_tchad.txt', 'r') as file:
    lines = file.readlines()
    for line in lines:
            linee = re.sub(regex, r'\n\n', line)
            print(linee)
            with open('arabe_tchad2.txt', 'a+') as file :
                file.write(re.sub(regex, "\n\n", line))"""




['Bariiʼa wa Chamaʼ humman kubaaraat al‑aayilaat al‑saakniin fi Ayyaluun.', 'Wa humman bas al‑tarado sukkaan Gaat.\n\nWa awlaad Bariiʼa humman Akhyu wa Chaachakh wa Yarimuut wa Zabadya wa Araad wa Adaar wa Mikaayiil wa Yichfa wa Yuuha.', '\n\n Wa awlaad Alifaaʼal humman Zabadya wa Machullam wa Hizkhi wa Haabar wa Yichmaraay wa Yizliiya wa Yubaab.\n\nWa awlaad Chimʼi humman Yakhiim wa Zikri wa Zabdi wa Aliʼinaay wa Siltaay wa Aliyiil wa Adaaya wa Baraaya wa Chimrat.\n\nWa awlaad Chaachakh humman Yichfaan wa Ibaar wa Aliyiil wa Abduun wa Zikri wa Hanaan wa Hananya wa Ilaam wa Antuutiya wa Yifadya wa Fanuwiil.\n\nWa awlaad Yaruuhaam humman Chamachraay wa Chaharya wa Atalya wa Yaʼrachya wa Iliiya wa Zikri.']
['Bariiʼa wa Chamaʼ humman kubaaraat al‑aayilaat al‑saakniin fi Ayyaluun.', 'Wa humman bas al‑tarado sukkaan Gaat.14‑16Wa awlaad Bariiʼa humman Akhyu wa Chaachakh wa Yarimuut wa Zabadya wa Araad wa Adaar wa Mikaayiil wa Yichfa wa Yuuha.', '17‑18 Wa awlaad Alifaaʼal humman Zabadya wa Ma

'with open(\'arabe_tchad.txt\', \'r\') as file:\n    lines = file.readlines()\n    for line in lines:\n            linee = re.sub(regex, r\'\n\n\', line)\n            print(linee)\n            with open(\'arabe_tchad2.txt\', \'a+\') as file :\n                file.write(re.sub(regex, "\n\n", line))'

 # Pour le français

In [ ]:
from scrapy import Selector
import requests

url = "https://www.biblegateway.com/passage/?search=Gen%C3%A8se%201&version=LSG"
html = requests.get(url).content   
sel = Selector(text=html) 
#text = sel.css('div.version-LSG result-text-style-normal text-html  p ::text').extract()  //class="version-LSG result-text-style-normal text-html"
text = sel.xpath('//div[contains(@class,"version-LSG result-text-style-normal text-html")]/p//text()').extract()
next_link = sel.xpath('//a[@class="next-link"]/@href').extract()

link = "https://www.biblegateway.com" + next_link[0] # Pour le prochain lien
print(link)
print("https://www.biblegateway.com/passage/?search=Gen%C3%A8se%202&version=LSG")

for i in range(1, len(text),2):
    print(text[i])

    


2025-03-26 15:24:34 [urllib3.connectionpool] DEBUG: Starting new HTTPS connection (1): www.biblegateway.com:443
2025-03-26 15:24:35 [urllib3.connectionpool] DEBUG: https://www.biblegateway.com:443 "GET /passage/?search=Gen%C3%A8se%201&version=LSG HTTP/1.1" 200 None


https://www.biblegateway.com/passage/?search=Gen%C3%A8se%202&version=LSG
https://www.biblegateway.com/passage/?search=Gen%C3%A8se%202&version=LSG
Au commencement, Dieu créa les cieux et la terre.
La terre était informe et vide: il y avait des ténèbres à la surface de l'abîme, et l'esprit de Dieu se mouvait au-dessus des eaux.
Dieu dit: Que la lumière soit! Et la lumière fut.
Dieu vit que la lumière était bonne; et Dieu sépara la lumière d'avec les ténèbres.
Dieu appela la lumière jour, et il appela les ténèbres nuit. Ainsi, il y eut un soir, et il y eut un matin: ce fut le premier jour.
Dieu dit: Qu'il y ait une étendue entre les eaux, et qu'elle sépare les eaux d'avec les eaux.
Et Dieu fit l'étendue, et il sépara les eaux qui sont au-dessous de l'étendue d'avec les eaux qui sont au-dessus de l'étendue. Et cela fut ainsi.
Dieu appela l'étendue ciel. Ainsi, il y eut un soir, et il y eut un matin: ce fut le second jour.
Dieu dit: Que les eaux qui sont au-dessous du ciel se rassemblent en

In [1]:
# Ici nous allons ecrire un code qui permet de recuperer toutes les pages du site web avec les methodes d'en haut

import scrapy
from scrapy.crawler import CrawlerProcess


class DCspider( scrapy.Spider ):  

    name = "dcspider"

    def start_requests( self ):    
        urls = "https://www.biblegateway.com/passage/?search=Gen%C3%A8se%201&version=LSG"
        yield scrapy.Request( url = urls, callback = self.parse )

        
    def parse( self, response ): 

        next_link = response.xpath('//a[@class="next-link"]/@href').extract() # Une partie du lien suivant extraite 
        print("---------------",next_link,"-----------------")
        link = "https://www.biblegateway.com" + next_link[0] # Pour le prochain lien(On complete la partie precedante avec le https..)
        
        text = response.xpath('//div[contains(@class,"version-LSG result-text-style-normal text-html")]/p//text()').extract() #Le text des verstes
       
        text = [t.strip() for t in text if t.strip()] # Pour enlever les espaces vides et les sauts de ligne ainsi que les balises html

        for i in range(1, len(text),2):         # On fait maintenant une boucle pour parcourir le text et ecrire les verstes dans un fichier
            with open("V_francais_3.txt","a+") as file:
                file.write(text[i])
                file.write("\n")
        yield response.follow( url = link, callback = self.parse )


process = CrawlerProcess() # Voila la derniere partie qui demarre le robot
process.crawl(DCspider)
process.start()


2025-05-04 14:10:56 [scrapy.utils.log] INFO: Scrapy 2.12.0 started (bot: scrapybot)
2025-05-04 14:10:56 [scrapy.utils.log] INFO: Versions: lxml 5.3.1.0, libxml2 2.12.9, cssselect 1.3.0, parsel 1.10.0, w3lib 2.3.1, Twisted 24.11.0, Python 3.12.3 (main, Feb  4 2025, 14:48:35) [GCC 13.3.0], pyOpenSSL 25.0.0 (OpenSSL 3.4.1 11 Feb 2025), cryptography 44.0.2, Platform Linux-6.11.0-17-generic-x86_64-with-glibc2.39
2025-05-04 14:10:56 [scrapy.addons] INFO: Enabled addons:
[]
2025-05-04 14:10:56 [scrapy.utils.log] DEBUG: Using reactor: twisted.internet.epollreactor.EPollReactor
2025-05-04 14:10:56 [scrapy.extensions.telnet] INFO: Telnet Password: 42c8a7e46a00c9a0
2025-05-04 14:10:56 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.corestats.CoreStats',
 'scrapy.extensions.telnet.TelnetConsole',
 'scrapy.extensions.memusage.MemoryUsage',
 'scrapy.extensions.logstats.LogStats']
2025-05-04 14:10:56 [scrapy.crawler] INFO: Overridden settings:
{}
2025-05-04 14:10:57 [scrapy.middlewa

--------------- ['/passage/?search=Gen%C3%A8se%202&version=LSG'] -----------------


2025-05-04 14:10:59 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.biblegateway.com/passage/?search=Gen%C3%A8se%202&version=LSG> (referer: https://www.biblegateway.com/passage/?search=Gen%C3%A8se%201&version=LSG)


--------------- ['/passage/?search=Gen%C3%A8se%203&version=LSG'] -----------------


2025-05-04 14:11:00 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.biblegateway.com/passage/?search=Gen%C3%A8se%203&version=LSG> (referer: https://www.biblegateway.com/passage/?search=Gen%C3%A8se%202&version=LSG)


...

--------------- ['/passage/?search=Apocalypse%2021&version=LSG'] -----------------


2025-05-04 14:25:31 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.biblegateway.com/passage/?search=Apocalypse%2021&version=LSG> (referer: https://www.biblegateway.com/passage/?search=Apocalypse%2020&version=LSG)


--------------- ['/passage/?search=Apocalypse%2022&version=LSG'] -----------------


2025-05-04 14:25:32 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.biblegateway.com/passage/?search=Apocalypse%2022&version=LSG> (referer: https://www.biblegateway.com/passage/?search=Apocalypse%2021&version=LSG)
2025-05-04 14:25:32 [scrapy.core.scraper] ERROR: Spider error processing <GET https://www.biblegateway.com/passage/?search=Apocalypse%2022&version=LSG> (referer: https://www.biblegateway.com/passage/?search=Apocalypse%2021&version=LSG)
Traceback (most recent call last):
  File "/home/koubra/.envIA/lib/python3.12/site-packages/scrapy/utils/defer.py", line 327, in iter_errback
    yield next(it)
          ^^^^^^^^
  File "/home/koubra/.envIA/lib/python3.12/site-packages/scrapy/utils/python.py", line 368, in __next__
    return next(self.data)
           ^^^^^^^^^^^^^^^
  File "/home/koubra/.envIA/lib/python3.12/site-packages/scrapy/utils/python.py", line 368, in __next__
    return next(self.data)
           ^^^^^^^^^^^^^^^
  File "/home/koubra/.envIA/lib/python3.12/

--------------- [] -----------------
